# Convert LabelMe Annotation to CSV-dataset

In [1]:
import json
import cv2 as cv
import numpy as np
import csv
import os
from os import walk
import random
from datetime import datetime

In [2]:
# Path the the folder containing the image annotations
#path = '../../03 Data/Simple Dataset/'
path = '../../03 Data/Single_Image/'
validation_percent = 0.2

#print(os.listdir(path))

all_files = os.listdir(path)

file_names = []
for file in all_files: 
    if file[-5:]  == '.json':
        file_names.append(file)
    
#print(file_names)
random.shuffle(file_names)
num_files = len(file_names)

val_files = file_names[0:int(num_files*validation_percent)]
train_files = file_names[int(num_files*validation_percent):]

print(len(file_names))
print(len(val_files))
print(len(train_files))

now = datetime.now()
date = now.strftime("%d-%m")

names = ['train_set_'+date+'.csv', 'val_set_'+date+'.csv']
combined_files = [train_files, val_files]

# create csv of train-set and val-set for feeding to the network
for i in range(len(names)):
    with open(names[i], 'w', newline='') as open_file:
        writer = csv.writer(open_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        with open('list_of_img_in_'+names[i], 'w') as f:
            for file in combined_files[i]:
                with open(path + file) as json_file:
                    data = json.load(json_file)
                    image_name = data['imagePath']
                    f.write("%s\n" % image_name)
                        # check if image contains annotations.
                    if len(data['shapes']) != 0:
                        for annotation in data['shapes']:
                                rot_rect = cv.minAreaRect(np.float32(annotation['points']))
                                bp = cv.boxPoints(rot_rect) # bp = box points
                                #p1 = [rot_rect[0]+]
                                # make sure that width and height are consistent
                                if rot_rect[-2][0] < rot_rect[-2][1]:
                                    rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
                                # make sure that no angle is above 90 or below -90
                                if rot_rect[-1] > 90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
                                if rot_rect[-1] < -90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
                                class_name = annotation['label']
                                writer.writerow([path + image_name, bp[0][0], bp[0][1], bp[1][0], bp[1][1],
                                                 bp[2][0], bp[2][1], bp[3][0], bp[3][1], class_name,rot_rect[-1]])
                                # format: img_path, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, class_name, angle
                    else:
                        writer.writerow([path + image_name, ",,,,,,,,,,"])
       

1
0
1
